In [1]:

import pandas as pd
import os
import shutil


In [2]:
# !pip install sunpy -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install drms -i https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
from prepare_base import PrepareWorker




### demo test ###
def demo_fun_use_Bptr(p_raw_path,t_raw_path,r_raw_path,sroot):
    # config
    # p_raw_path = r"/media/zander/Data/now/work-inner/sun/LINFF/data/hmi.sharp_cea_720s.377.20110215_020000_TAI.Bp.fits"
    # t_raw_path = r"/media/zander/Data/now/work-inner/sun/LINFF/data/hmi.sharp_cea_720s.377.20110215_020000_TAI.Bt.fits"
    # r_raw_path = r"/media/zander/Data/now/work-inner/sun/LINFF/data/hmi.sharp_cea_720s.377.20110215_020000_TAI.Br.fits"
    # sroot=r"/public1/home/sc81826/temp/out"
    # create object
    os.makedirs(sroot,exist_ok=True)
    shutil.copyfile(p_raw_path,sroot+"/Bp.fits")
    shutil.copyfile(r_raw_path,sroot+"/Br.fits")
    shutil.copyfile(t_raw_path,sroot+"/Bt.fits")
    pre_worker = PrepareWorker()
    # pre
    pre_worker.prepare_from_fits_Bprt(p_raw_path,t_raw_path,r_raw_path,sroot)
    # print("finish demo_fun_use_Bptr_local")



def deal_one(sample):

    bp=job_root+sample["bp"]
    bt=job_root+sample["bt"]
    br=job_root+sample["br"]
    HARP_NUM=sample["HARP_NUM"]
    HARPNUM_TREC=sample["HARPNUM_TREC"]
    save_p=os.path.join(save_root,str(HARP_NUM),"hmi.sharp_cea_720s.{}".format(HARPNUM_TREC))
    demo_fun_use_Bptr(p_raw_path=bp,t_raw_path=bt,r_raw_path=br,sroot=save_p)


In [4]:
job_path="/home/bingxing2/home/scx6069/zzr/data/nlfff_append/nlfff_need_append_202404160610.csv"
job_root="/home/bingxing2/home/scx6069/zzr/data/nlfff_append/content/jsoc1.stanford.edu"
save_root="/home/bingxing2/home/scx6069/zzr/data/nlfff_append/product"
jobscsv=pd.read_csv(job_path)


In [5]:
len(jobscsv)

33536

In [6]:

# this_id_list=list(range(len(jobscsv)))

# sucess=[]
# fail=[]
# i=0
# for jobid in this_id_list:
#     sample=jobscsv.loc[jobid]
#     try:
#         deal_one(sample)
#         sucess.append(jobid)
#     except BaseException as e:
#         print(e)
#         sucess.append(jobid)
#     i+=1
#     if i%100==0:
#         print(i)
#     # break
    

In [7]:
import multiprocessing
import pandas as pd

# 假设这是你的任务处理函数
# def deal_one(sample):
#     # 处理任务的代码
#     pass

# # 假设这是你的任务列表
# jobscsv = pd.DataFrame(...)  # 假设这里是你的任务数据

# 定义处理任务的函数
def process_job(jobid):
    sample = jobscsv.loc[jobid]
    try:
        deal_one(sample)
        return jobid, True  # 处理成功
    except Exception as e:
        print(f"Error processing job {jobid}: {e}")
        return jobid, False  # 处理失败

if False: #__name__ == "__main__":
    this_id_list = list(range(len(jobscsv)))
    pool = multiprocessing.Pool(processes=100)  # 创建进程池，最多同时运行100个进程
    results = pool.map(process_job, this_id_list)  # 并发处理任务列表
    pool.close()
    pool.join()

    # 处理处理成功和失败的任务
    success = [jobid for jobid, status in results if status]
    failure = [jobid for jobid, status in results if not status]

    print(f"Successfully processed {len(success)} jobs.")
    print(f"Failed to process {len(failure)} jobs.")


In [1]:
import pandas as pd
import random

# 读取CSV文件
job_path="/home/bingxing2/home/scx6069/zzr/data/nlfff_append/run2/nlfff_need_append_202404160610.csv"

df = pd.read_csv(job_path)

# 随机打乱数据
random.seed(123)  # 设置随机种子，保证每次结果一致
df_shuffled = df.sample(frac=1).reset_index(drop=True)

# 分成part个部分
part=16
total_rows = len(df_shuffled)
part_size = total_rows // part
parts = [df_shuffled[i*part_size:(i+1)*part_size] for i in range(part)]

# 保存分割后的数据
for i, part_df in enumerate(parts):
    part_df.to_csv(job_path+f'.part_{i+1}.csv', index=False)
